In [1]:
import nibabel as nib
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import glob

The colab read data from drive: you need to give access by clicking in the link

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [0]:
#DIR = "BrainHack/"

#DIR = "/content/drive/My Drive/BrainHack/"


In [6]:
labels = pd.read_csv(DIR + 'dataset.csv', delimiter = ',', header = 0)
print(labels.shape)
labels.head()
#sum(labels['COR'] ==1)

(727, 19)


,Id,CT,MR,GE,Siemens,Philips,Toshiba,CTA,NCTT,T2W,T1W,DWI,FLAIR,GRE,1_5,3,AX,SAG,COR
0,8B11_2_MDHENA10480B0_CTGE_OPCTALR_CTA_H_N_DR_3...,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
1,8B11_2_MDHENA10484B0_CTGE_OPCTALR_CTA_2.5_2018...,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
2,8B11_2_MDHENA10485B0_CTGE_OPCTALR_Ax_CTA_5.0_2...,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
3,8B11_2_MDHENA10494B0_CTGE_OPCTALR_CTA_2.5_2018...,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
4,8B11_2_MDHENA10526B0_CTGE_OPCTALR_CTA_H_N-1.25...,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0


In [7]:
def find_nii(DIR):
    files_CT = glob.glob(os.path.join(DIR,'*','*','*','*','*.nii'))
    print(len(files_CT))
    print(os.path.join(DIR,'MRI','*','*','*','*','*.nii'))
    files_MR = glob.glob(os.path.join(DIR,'MRI','*','*','*','*','*.nii'))
    print(len(files_MR))

    return files_CT,files_MR

files_CT2,files_MR2 = find_nii(DIR)

164
/content/drive/My Drive/BrainHack/MRI/*/*/*/*/*.nii
563


In [0]:
files_MR = glob.glob(os.path.join(DIR,'MRI','*','*','*','*','*.nii'))

In [0]:
files_MR_name = []
for files_MR_element in files_MR:
    files_MR_name.append((files_MR_element.rsplit('/', 1)[-1])[:-4])

In [10]:
len(files_MR_name)

563

In [11]:
label_set = pd.DataFrame(columns = labels.columns)
for name_idx, name in enumerate(files_MR_name):
    temp = np.where(labels["Id"] == name)[0][0]
    #print(temp)
    #print(name_idx)
    label_set.loc[name_idx] = (labels.iloc[temp,:])
label_set.shape

(563, 19)

In [12]:
files_MR[0:4]

['/content/drive/My Drive/BrainHack/MRI/FLAIR/Philips/1_5/8B11_2_MDHENA1011824h_MRPHILIPS1_5_OPFLAIR2DLR/8B11_2_MDHENA1011824h_MRPHILIPS1_5_OPFLAIR2DLR_AX_FLAIR_CLEAR_20171216104550_501.nii',
 '/content/drive/My Drive/BrainHack/MRI/FLAIR/Philips/1_5/8B11_2_MDHENA1013324h_MRPHILIPS1_5_OPFLAIR2DLR/8B11_2_MDHENA1013324h_MRPHILIPS1_5_OPFLAIR2DLR_AX_FLAIR_MV_20180104101932_401.nii',
 '/content/drive/My Drive/BrainHack/MRI/FLAIR/Philips/1_5/8B11_2_MDHENA1017924h_MRPHILIPS1_5_OPFLAIR2DLR/8B11_2_MDHENA1017924h_MRPHILIPS1_5_OPFLAIR2DLR_AX_FLAIR_CLEAR_20180220124258_501.nii',
 '/content/drive/My Drive/BrainHack/MRI/FLAIR/Philips/1_5/8B11_2_MDHENA10053FU_MRPHILIPS1_5_OPFLAIR2DLR/8B11_2_MDHENA10053FU_MRPHILIPS1_5_OPFLAIR2DLR_AX_FLAIR_CLEAR_20171009160524_501.nii']

In [13]:
files = files_MR[10]
x = (nib.load(files).get_data())
print(x.shape)
files
enumerate(files_MR)
label_set["Id"][1]

(384, 384, 27)


'8B11_2_MDHENA1013324h_MRPHILIPS1_5_OPFLAIR2DLR_AX_FLAIR_MV_20180104101932_401'

In [0]:
raw_dataset = []
raw_dataset_label = pd.DataFrame(columns = labels.columns)
raw_dataset_label_idx = 0

In [15]:
for file_idx, _ in enumerate(files_MR):
    temp_file = nib.load(files_MR[file_idx], keep_file_open = False)
    current_MRI_3D = temp_file.get_data()
    temp_file.uncache()
    #del temp_file
    print("\rOpened the archive number {} of {}". format(file_idx, len(files_MR)))
    
    if current_MRI_3D.ndim == 4:
        for i in range(current_MRI_3D.shape[2]):
            for j in range(current_MRI_3D.shape[3]):
                raw_dataset.append(current_MRI_3D[:,:,i,j])
                raw_dataset_label.loc[raw_dataset_label_idx] = label_set.loc[file_idx]
                raw_dataset_label["Id"][raw_dataset_label_idx] = raw_dataset_label["Id"][raw_dataset_label_idx] + str(i) + "_" + str(j)
                raw_dataset_label_idx = raw_dataset_label_idx + 1
                
    elif current_MRI_3D.ndim == 3:
        for i in range(current_MRI_3D.shape[2]):
                raw_dataset.append(current_MRI_3D[:,:,i])
                raw_dataset_label.loc[raw_dataset_label_idx] = label_set.loc[file_idx]
                raw_dataset_label["Id"][raw_dataset_label_idx] = raw_dataset_label["Id"][raw_dataset_label_idx] + str(i) + "_0"
                raw_dataset_label_idx = raw_dataset_label_idx + 1
    
    
    
    


Opened the archive number 0 of 563
Opened the archive number 1 of 563
Opened the archive number 2 of 563
Opened the archive number 3 of 563
Opened the archive number 4 of 563
Opened the archive number 5 of 563
Opened the archive number 6 of 563
Opened the archive number 7 of 563
Opened the archive number 8 of 563
Opened the archive number 9 of 563
Opened the archive number 10 of 563
Opened the archive number 11 of 563
Opened the archive number 12 of 563
Opened the archive number 13 of 563
Opened the archive number 14 of 563
Opened the archive number 15 of 563
Opened the archive number 16 of 563
Opened the archive number 17 of 563
Opened the archive number 18 of 563
Opened the archive number 19 of 563
Opened the archive number 20 of 563
Opened the archive number 21 of 563
Opened the archive number 22 of 563
Opened the archive number 23 of 563
Opened the archive number 24 of 563
Opened the archive number 25 of 563
Opened the archive number 26 of 563
Opened the archive number 27 of 563
Op

In [4]:
raw_dataset.shape

NameError: name 'raw_dataset' is not defined